In [2]:
import pandas as pd
import requests

your_auth_token = 'a69cbf86d6c89f04112abace5d954c30'
ACCOUNT_SID = 'ACeaf54618524c574255b0d98c9385e82c'

In [3]:
connected_df = pd.read_csv("connected_twilio.csv")
disconnected_df = pd.read_csv("disconnected_twilio.csv")
connected2_df = pd.read_csv("../GetPhoneInfo/connected_final.csv")
disconnected2_df = pd.read_csv("../GetPhoneInfo/disconnected_final.csv")

In [4]:
connected_combo = pd.merge(connected_df, connected2_df, on = 'OFFICE_TELEPHONE', suffixes = ('_Twilio','_GetPhoneInfo'))
disconnected_combo = pd.merge(disconnected_df, disconnected2_df, on = 'OFFICE_TELEPHONE', suffixes = ('_Twilio','_GetPhoneInfo'))

In [6]:
connnected_combo = connected_combo[['OFFICE_TELEPHONE','Name_Twilio', 'Name_GetPhoneInfo', 'PHYSICIAN_FIRST_NAME', 'PHYSICIAN_LAST_NAME']]

In [6]:
disconnected_combo[['OFFICE_TELEPHONE','Caller Type', 'PhoneType','Name_Twilio', 'Name_GetPhoneInfo', 'PHYSICIAN_FIRST_NAME', 'PHYSICIAN_LAST_NAME']]

,OFFICE_TELEPHONE,Caller Type,PhoneType,Name_Twilio,Name_GetPhoneInfo,PHYSICIAN_FIRST_NAME,PHYSICIAN_LAST_NAME
0,7848098180,Not Valid,NaN,Not Valid,NaN,ROBERTO,ALVAREZ-SWIHART
1,2123047330,BUSINESS,BUSINESS,NYPRES,NY PRESBYTERIAN,MYKOLA,ALYSKEWYCZ
2,4017665959,BUSINESS,BUSINESS,CARDIOLOGY ASSOCIATES,CARDIOLOGY ASSOCIATES,MOHAMMAD,ARIF
3,9014486657,BUSINESS,BUSINESS,UT HEALTHSCICTR,UNIV OF TN HSC,WILLIAM,BASTNAGEL
4,4156412140,BUSINESS,BUSINESS,ST. LUKE'S WOME,SAINT LUKES WOMENS CENTER,YAMILEE,BERMINGHAM
5,7273020000,BUSINESS,RESIDENTIAL,FL INS SVC,L BHATTACHARJEE,LAKSHMISRI,BHATTACHARJEE
6,2196638110,NaN,BUSINESS,NaN,BLEZA ROBERT D MD,ROBERT,BLEZA
7,9736230606,NaN,BUSINESS,NaN,V D BODAS MD CLINICS MEDICAL,MONA,BODA
8,2514357223,BUSINESS,BUSINESS,MOBILE INFIRMA,MOBILE INFIRMAR,AHDY,BOLIS
9,4153522161,NaN,BUSINESS,NaN,PRESTIGE INTERNATIONAL,SANDHYA,BONDADA


In [8]:
disconnected_combo = disconnected_combo.rename(columns={"Caller Type": "Caller_Type"})
connected_combo = connected_combo.rename(columns={"Caller Type": "Caller_Type"})

In [19]:
disconnected_combo = disconnected_combo.fillna('nada')
connected_combo = connected_combo.fillna('nada')

In [34]:
def is_healthy(name):
    med_words = ['radiology','glaucoma', 'dr ','ortho', 'surgi','cancer', 'cardiology', 'neurology', 'family care', 'pulmonary', 'anesthesia', 'orthope', 'urgent care', 'allergy', 'kidney', 'surgery', 'hosp', 'mri', 'throat', 'dentist', 'med', 'clinic', 'health', 'gastroenter','anesthesiologist','patient','physician','surgeon','doctor','hospital', 'md', 'medical', 'pediatrics', 'm.d.']
    healthy = False
    for word in med_words:
        if word in str(name).lower():
            healthy = True   
    return(healthy)

In [51]:
def analyze_twilio(dataframe):
    name_match = 0
    healthy = 0
    business = 0
    consumer = 0 
    no_name = 0
    
    for row in dataframe.itertuples():
        if str(row.PHYSICIAN_LAST_NAME).lower() in str(row.Name_Twilio).lower():
            name_match +=1
        if is_healthy(row.Name_Twilio) == True:
            healthy += 1
#         if is_healthy(row.Name_Twilio) == False:
#             if row.Name_Twilio != 'nada':
#                 print(row.Name_Twilio)
#             if is_healthy(row.Name_GetPhoneInfo) == True:
#                 print(f'{row.Name_Twilio} is right and {row.Name_GetPhoneInfo} is wrong. Last name is {row.PHYSICIAN_LAST_NAME}')
        if row.Caller_Type == 'BUSINESS':
            business += 1
        if row.Caller_Type == 'CONSUMER':
            consumer += 1
        if row.Name_Twilio == 'nada' or row.Name_Twilio == 'Not Valid' or row.Name_Twilio == 'No Name':
            no_name += 1
#         if str(row.Name_GetPhoneInfo).lower() not in str(row.Name_Twilio).lower():
#             print(f'{row.Name_Twilio} is not {row.Name_GetPhoneInfo}')
    new_dict = {
        'No Name': no_name,
        'Exact Name Matches': name_match,
        'Relevant Names': healthy,
        'Business Phone': business,
        'Consumer Phone': consumer
        }
        
    return(new_dict)

In [52]:
analyze_twilio(connected_combo)

{'No Name': 29,
 'Exact Name Matches': 5,
 'Relevant Names': 45,
 'Business Phone': 66,
 'Consumer Phone': 5}

In [53]:
analyze_twilio(disconnected_combo)

{'No Name': 40,
 'Exact Name Matches': 7,
 'Relevant Names': 27,
 'Business Phone': 53,
 'Consumer Phone': 7}

In [58]:
ekata_connected = pd.read_csv("../Ekata/ekata_connected.csv")
ekata_disconnected = pd.read_csv("../Ekata/ekata_disconnected.csv")
ekata_twilio_conn = pd.merge(ekata_connected, connected_combo, on='OFFICE_TELEPHONE', suffixes = ('_Ekata', '_GetPhoneInfo'))



,Address_Ekata,AlternatePhone,Carrier,City_Ekata,Commercial,Date_Ekata,Error,Industry,LineType,Name,...,QualityScore,State_GetPhoneInfo,Zipcode,PHYSICIAN_FIRST_NAME,PHYSICIAN_LAST_NAME,OFFICE_ADDRESS_LINE_2,OFFICE_ADDRESS_CITY,OFFICE_ADDRESS_STATE,OFFICE_ADDRESS_ZIP,COMMENTS
0,320 Abington Dr Ste 4,None,Comcast,Reading,True,11/19/2016,None,Ambulatory Health Care Services,FixedVOIP,Timothy C Stringer DPM,...,HIGH,PA,19610,LAURA,STRINGER,1981 STATE HILL ROAD,WYOMISSING,PA,19610,COMPLETE
1,2854 Bell St,None,Time Warner Communications,Zanesville,True,2/28/2019,None,Ambulatory Health Care Services,FixedVOIP,"Toth, Randall S",...,HIGH,OH,43701,VANESSA,FALK,2854 BELL ST,ZANESVILLE,OH,43701,COMPLETE
2,3901 Central Pike Ste 351,16156013014,Comcast,Hermitage,True,11/20/2016,None,Ambulatory Health Care Services,FixedVOIP,Gowda Ear Nose & Throat Clinic,...,HIGH,TN,37076,JUSTIN,MORGAN,3901 CENTRAL PIKE,HERMITAGE,TN,37076,COMPLETE
3,6601 Lyndale Ave S Ste 220,17635440696,Comcast,Minneapolis,True,11/17/2016,None,Ambulatory Health Care Services,FixedVOIP,"Esten, Andrew MD",...,LOW,MN,55423,ANDREW,ESTEN,6601 LYNDALE AVE S STE 220,MINNEAPOLIS,MN,55423,COMPLETE
4,1215 S Coulter St Ste 403,18066772001,Pathwayz Communications,Amarillo,True,11/17/2016,None,Ambulatory Health Care Services,Landline,Amarillo Medical Spec Llp,...,HIGH,TX,79106,RAGHAVENDRA,SARALAYA,1215 S COULTER ST STE 403,AMARILLO,TX,79106,COMPLETE
5,121 Cahill Rd Ste 204,18002454362,CenturyLink,Branson,True,11/18/2016,None,Ambulatory Health Care Services,Landline,Branson Neurology & Pain Center,...,HIGH,MO,65616,AHMED,ROBBIE,121 CAHILL RD,BRANSON,MO,65616,COMPLETE
6,10721 Main St Ste 1300,None,Level 3 Communications,Fairfax,True,11/17/2016,None,Ambulatory Health Care Services,NonFixedVOIP,Allen & Forrest,...,LOW,VA,22030,SCOTT,FORREST,10721 MAIN ST STE 1300,FAIRFAX,VA,22030,COMPLETE
7,645 Marble Ave,None,Cablevision Lightpath,Thornwood,True,11/19/2016,None,Ambulatory Health Care Services,FixedVOIP,Margaret M Andersen MD,...,HIGH,NY,10594,MEGHAN,AUTEN,465 COLUMBUS AVE,VALHALLA,NY,10595,COMPLETE
8,10707 66th St N Ste A,None,Bright House Networks Information Services (Fl...,Pinellas Park,True,11/15/2016,None,Ambulatory Health Care Services,FixedVOIP,Sarah Anne Digby DO,...,HIGH,FL,33782,SARA,DOWNS,10707 66TH ST N STE A,PINELLAS PARK,FL,33782,COMPLETE
9,3301 Alta Arden Expy Ste 3,None,Bandwidth SMSEnabled,Sacramento,True,11/15/2016,None,Ambulatory Health Care Services,NonFixedVOIP,William Kenneth Mora MD,...,HIGH,CA,95825,MICHAEL,KWIKER,3301 ALTA ARDEN EXPY STE 3,SACRAMENTO,CA,95825,COMPLETE


In [97]:
ekata_twilio_conn.to_csv('ekata_twilio_con.csv')
ekata_twilio_disconn.to_csv('ekata_twilio_discon.csv')

In [56]:
analyze_twilio(ekata_twilio_conn)

{'No Name': 14,
 'Exact Name Matches': 3,
 'Relevant Names': 15,
 'Business Phone': 23,
 'Consumer Phone': 3}

In [59]:
ekata_twilio_disconn = pd.merge(ekata_disconnected, disconnected_combo, on='OFFICE_TELEPHONE', suffixes = ('_Ekata', '_GetPhoneInfo'))
analyze_twilio(ekata_twilio_disconn)

{'No Name': 17,
 'Exact Name Matches': 4,
 'Relevant Names': 7,
 'Business Phone': 19,
 'Consumer Phone': 4}

In [1]:
connected_combo

NameError: name 'connected_combo' is not defined

In [92]:
def twilio_cnam(number):
    url = f'https://lookups.twilio.com/v1/PhoneNumbers/+1{number}?AddOns=telo_opencnam'
    response =  requests.get(url, auth=(ACCOUNT_SID, your_auth_token))
    results = response.json()
    try:
        name = results['add_ons']['results']['telo_opencnam']['result']['name']
    except:
        name = "No Name"
    return(name)

In [89]:
def twilio_cnam_test(dataframe):
    name_list = []
    for row in dataframe.itertuples():
        new_dict = {}
        number = row.OFFICE_TELEPHONE
        new_dict["OFFICE_TELEPHONE"] = number
        name = twilio_cnam(number)
        new_dict["Name_CNAM"] = name
        new_dict["Name_WSLive"] = f'{row.PHYSICIAN_FIRST_NAME} {row.PHYSICIAN_LAST_NAME}'
        new_dict["Name_GetPhoneInfo"] = row.Name_GetPhoneInfo
        new_dict["Name_Twilio"] = row.Name_Twilio
        name_list.append(new_dict)
    return(name_list)

In [84]:
names_listicle  = twilio_cnam_test(connected_combo)

OHIOHEALTH
MCBRIDE ORTHOPE
PAYAM YASHAR MD
PSS PELHAM
CASTROVILLE, TX
LAHEY,HOSPITAL
MADISON IRVING
HACKENSACK, NJ
DR TIMOTHY C ST
ORTHOPEDIC
ALLERGY AND ENT
KIDNEY SPECIALI
AMA MEDICAL SPC
BRANSON, MO
ALLEN, TERRY DG
WESTCHESTER HEA
PINELLAS INTERN
SACRAMENTO, CA
WARREN THOMAS L
HENDERSON, NV
CIGNA CORP
HILLCREST HOSP
COOMBS SETH
BACHARACH INSTI
AHADI SUE MD
CORVALLIS, MT
NEWPORT BEA, CA
ADMIN SUPPORT B
BROOKLYN, NY
HOWANITZ E PAUL
LEE DAVIS MEDIC
UNIVERSITY
NEA BAPT CLINIC
NORTON MED ASSO
SOUTH CENTRAL
LYNN EYE SURGER
MOUNTAIN TO, PA
GRAND STRAND RE
BIDH-PLYMOUTH
PAVILION FMLY P
PRESBYTERIAN
KNEE, HIP AND S
TEXARKANA, TX
MINNEAPOLIS, MN
PIH HEALTH
ANNE ARUNDEL ME
MIHS MAIN
HEALTHCARE PART
STILLWATER, MN
AHS OVERLOOK
AV CARDIOLOGY
ADVOCATE HEALTH
ASSOCIATES IN E
BETH, ISRAEL DE
ORTHOPEDIC CENT
CHILDREN S,HOSP
EL PASO CANCER
GROSSMONT IMAGI
METHODISTFAMILY
ELIZABETHRUSS .
CHRISTIANACAREV
CASS REGIONAL
NORTH DALLAS PL
CLEVELAND RD FA
SINAI HLTH SYST
SOUTHERN ORTHO
SW MED CTR
SUMMA WESTE

In [87]:
connected_names = pd.DataFrame(names_listicle)
connected_names

,Name_CNAM,OFFICE_TELEPHONE
0,OHIOHEALTH,6147884440
1,MCBRIDE ORTHOPE,4052309435
2,PAYAM YASHAR MD,3105562020
3,PSS PELHAM,8648499555
4,"CASTROVILLE, TX",8305383550
5,"LAHEY,HOSPITAL",7817448046
6,MADISON IRVING,3154712646
7,"HACKENSACK, NJ",2014883131
8,DR TIMOTHY C ST,6106702277
9,ORTHOPEDIC,7404543273


In [78]:
twilio_cnam('8567577799')

'COOPER HEALTHCA'

In [88]:
names_listicle

[{'OFFICE_TELEPHONE': 6147884440, 'Name_CNAM': 'OHIOHEALTH'},
 {'OFFICE_TELEPHONE': 4052309435, 'Name_CNAM': 'MCBRIDE ORTHOPE'},
 {'OFFICE_TELEPHONE': 3105562020, 'Name_CNAM': 'PAYAM YASHAR MD'},
 {'OFFICE_TELEPHONE': 8648499555, 'Name_CNAM': 'PSS PELHAM'},
 {'OFFICE_TELEPHONE': 8305383550, 'Name_CNAM': 'CASTROVILLE, TX'},
 {'OFFICE_TELEPHONE': 7817448046, 'Name_CNAM': 'LAHEY,HOSPITAL'},
 {'OFFICE_TELEPHONE': 3154712646, 'Name_CNAM': 'MADISON IRVING'},
 {'OFFICE_TELEPHONE': 2014883131, 'Name_CNAM': 'HACKENSACK, NJ'},
 {'OFFICE_TELEPHONE': 6106702277, 'Name_CNAM': 'DR TIMOTHY C ST'},
 {'OFFICE_TELEPHONE': 7404543273, 'Name_CNAM': 'ORTHOPEDIC'},
 {'OFFICE_TELEPHONE': 6158898802, 'Name_CNAM': 'ALLERGY AND ENT'},
 {'OFFICE_TELEPHONE': 6128238001, 'Name_CNAM': 'KIDNEY SPECIALI'},
 {'OFFICE_TELEPHONE': 8066772002, 'Name_CNAM': 'AMA MEDICAL SPC'},
 {'OFFICE_TELEPHONE': 4173357222, 'Name_CNAM': 'BRANSON, MO'},
 {'OFFICE_TELEPHONE': 7033856870, 'Name_CNAM': 'ALLEN, TERRY DG'},
 {'OFFICE_TELEPHO

In [93]:
disconnected_names = twilio_cnam_test(disconnected_combo)
disconnected_names_df = pd.DataFrame(disconnected_names)

disconnected_names_df

In [95]:
disconnected_names_df.to_csv('names.csv')

In [8]:
connected_names = pd.read_csv('connected_names.csv')
connected_names.head()

,Name_CNAM,OFFICE_TELEPHONE
0,OHIOHEALTH,6147884440
1,MCBRIDE ORTHOPE,4052309435
2,PAYAM YASHAR MD,3105562020
3,PSS PELHAM,8648499555
4,"CASTROVILLE, TX",8305383550


In [23]:
disconnected_names_all= pd.read_csv("disconnected_names.csv")

In [9]:
connected_names_all = pd.merge(connected_names, connected_combo, on = 'OFFICE_TELEPHONE')

In [10]:
def generate_names_table(dataframe):
    name_list = []
    for row in dataframe.itertuples():
        new_dict = {}
        number = row.OFFICE_TELEPHONE
        new_dict["OFFICE_TELEPHONE"] = number
        new_dict["Name_CNAM"] = row.Name_CNAM
        new_dict["Name_WSLive"] = f'{row.PHYSICIAN_FIRST_NAME} {row.PHYSICIAN_LAST_NAME}'
        new_dict["Name_GetPhoneInfo"] = row.Name_GetPhoneInfo
        new_dict["Name_Twilio"] = row.Name_Twilio
        name_list.append(new_dict)
    names_table = pd.DataFrame(name_list)
    return(names_table)

In [11]:
connected_names_df = generate_names_table(connected_names_all)

In [14]:
connected_names_df.to_csv('connected_names.csv')

In [15]:
ekata_connected_df = pd.read_csv("../Ekata/ekata_connected.csv")

In [19]:
EKATA_CONNECTED = ekata_connected_df[['Name', 'OFFICE_TELEPHONE']]

In [21]:
connected_name_comparison = pd.merge(EKATA_CONNECTED, connected_names_df, on = 'OFFICE_TELEPHONE')

In [25]:
ekata_disconnected_df = pd.read_csv("../Ekata/ekata_disconnected.csv")
EKATA_DISCONNECTED = ekata_disconnected_df[['Name', 'OFFICE_TELEPHONE']]
disconnected_name_comparison = pd.merge(EKATA_DISCONNECTED, disconnected_names_all, on = 'OFFICE_TELEPHONE')
disconnected_name_comparison

,Name,OFFICE_TELEPHONE,Name_CNAM,Name_GetPhoneInfo,Name_Twilio,Name_WSLive
0,NaN,5857773511,DEPAUL MT HOPE,DEPAUL M HOPE,DEPAUL MT HOPE,KIMBERLY ERWAY
1,Lisete R Fink,6104199092,BOYS AND GIRLS,S GIRLY,BOYS AND GIRLS,CATHERINE DURISHIN
2,Sonja H Frazier,7273020000,"PASADENA, FL",L BHATTACHARJEE,FL INS SVC,LAKSHMISRI BHATTACHARJEE
3,Christine Mary Kelley,8585739902,"SAN DIEGO, CA",G SRVS,GERINET DR SRVS,AIREEN GUTIERREZ
4,Justin A Roberts,6194027300,"ROBERTS,JUSTIN",JUSTIN A ROBERTS,JUSTIN ROBERTS,PATRICIA PISINGER
5,Jose Luis Lezama MD,8139727629,"UNIVERSITY , FL",GSA DEPT OF VA,DEPT OF VETERAN,SURINDER GILL
6,"Young, Allan MD",5164839020,"HEMPSTEAD, NY",ALLAN YOUNG MD PHYSICIANS GENERAL,ALLAN YOUNG,ALLAN YOUNG
7,Uab Hospital,2059393361,"BIRMINGHAM, AL",UAB,UAB,ALICIA LEADFORD
8,Stephen J Massicotte Pc,3172808410,"INDIANAPOLI, IN",MASSICOTTE STEPHEN J MD PC,MASSICOTTE STEP,STEPHEN MASSICOTTE
9,Mervyn H Woolf MD,4017267300,"PAWTUCKET, RI",ANESTHESIA CARE INC,ANESTHESIA CARE,SUSAN WUTHRICH


In [27]:
connected_name_comparison.to_csv('connected_name_comparison.csv')
disconnected_name_comparison.to_csv('disconnected_name_comparison.csv')